# Imports

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
ls

# IMDB Database

In [ ]:
conn = sqlite3.connect('./zippedData/im.db/im.db')

In [ ]:
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res.fetchall():
    print(name[0])

In [ ]:
pd.read_sql("""
SELECT
    genres
FROM 
    movie_basics
;""", conn).describe()

In [ ]:
pd.read_sql("""
SELECT
    genres
FROM 
    movie_basics
;""", conn).value_counts()

In [ ]:
pd.read_sql("""
SELECT
    averagerating
FROM 
    movie_ratings
;""", conn).describe()

# Box Office Mojo

- Removed nulls for domestic_gross, converted to float.
- Same for foreign_gross
- Created new column ('total_gross') by adding domestic_gross and foreign_gross, concerted to int64
- Sorted by total_gross
- Turns out it's a lot of kids movies mixed in.  Decided to go with box office data from the Numbers as well to avoid confusion

In [ ]:
df_bom = pd.read_csv('./zippedData/bom.movie_gross.csv.gz')
df_bom.index

In [ ]:
df_bom.head()

In [ ]:
df_bom['domestic_gross'].isna().sum()

In [ ]:
type(df_bom['domestic_gross'][1])

In [ ]:
df_bom['domestic_gross'] = df_bom['domestic_gross'].fillna(value='0')

In [ ]:
df_bom['domestic_gross'] = df_bom['domestic_gross'].astype('int64')
df_bom.head()

In [ ]:
df_bom['domestic_gross'].isna().sum()

In [ ]:
type(df_bom['domestic_gross'][0])

In [ ]:
df_bom['foreign_gross'].isna().sum()

In [ ]:
df_bom['foreign_gross'] = df_bom['foreign_gross'].fillna(value='0')

In [ ]:
df_bom.isna().sum()

In [ ]:
df_bom['foreign_gross'] = df_bom['foreign_gross'].str.replace(',','')
df_bom.head()

In [ ]:
df_bom['foreign_gross'] = df_bom['foreign_gross'].astype('float64')
df_bom.head()

In [ ]:
type(df_bom['foreign_gross'][0])

In [ ]:
df_bom['total_gross'] = df_bom['domestic_gross'] + df_bom['foreign_gross']
df_bom.head()

In [ ]:
type(df_bom['total_gross'][1])

In [ ]:
df_bom['total_gross'] = df_bom['total_gross'].astype('int64')
df_bom.head(11)

In [ ]:
df_bom.index

In [ ]:
df_bom.sort_values('total_gross', ascending=False)
df_bom.head(11)

# Rotten Tomatoes Movie Index

Tried to associate genre and box office, but majority (>1200 out of 1500+) of box office were nulls

In [ ]:
df_rt_movie = pd.read_csv('./zippedData/rt.movie_info.tsv.gz', delimiter="\t")
df_rt_movie.index

In [ ]:
df_rt_movie.head()

In [ ]:
df_rt_movie.info()

In [ ]:
df_rt_movie.isna().sum()

In [ ]:
df_rt_movie_clean = df_rt_movie.dropna(subset=['box_office'])
df_rt_movie_clean.head()

In [ ]:
df_rt_movie_clean.info()

In [ ]:
df_rt_movie_clean['total_box_office'] = df_rt_movie_clean['box_office'].str.replace(',','')
df_rt_movie_clean.head()

In [ ]:
df_rt_movie_clean.info()

# Rotten Tomatoes Reviews

Avoided this database, preferring viewer ratings / user counts / 'popularity' in TMDB

Contains: id, review, rating, fresh, critic, top_critic, publisher, date

In [ ]:
df_rt_reviews = pd.read_csv('./zippedData/rt.reviews.tsv.gz', encoding='windows-1252', delimiter="\t")
df_rt_reviews.index

In [ ]:
df_rt_reviews.head()

# The Movie Data Base

- Able to sort by highest vote average, with conditional of movies with >10,000 vote counts and avg >7
- Then associated that to genre_ids category via color-coding in plot
- Found short list of what numbers in genre_ids translate too.  Did not investigate API for full list.  Started replacing genre ID numbers from that list, but decided it created too congested a dataframe
- Started comparisons using TMDB's 'popularity' column (>40) and 'vote_count' (>1000) at the end

In [ ]:
df_tmdb = pd.read_csv('./zippedData/tmdb.movies.csv.gz')
df_tmdb.index

In [ ]:
df_tmdb.head()

In [ ]:
top_vote_avg = df_tmdb.sort_values('vote_average', ascending=False)
top_vote_avg

In [ ]:
type(top_vote_avg['vote_count'][0])

In [ ]:
type(top_vote_avg['vote_average'][0])

In [ ]:
top_votes = top_vote_avg.loc[(top_vote_avg['vote_count'] > 10000) & (top_vote_avg['vote_average'] > 7)]
top_votes = top_votes.sort_values('vote_average', ascending=False)

In [ ]:
top_votes = top_votes.drop_duplicates(subset='title')

In [ ]:
top_votes.head()

In [ ]:
top_20_votes = top_votes[:20]
top_20_votes

In [ ]:
sns.set_theme(style='ticks')
sns.catplot(x='vote_average',y='original_title',kind='bar',data=top_20_votes)
figsize=(15,6)
plt.show();

In [ ]:
# code for plot used on Action/Adventure slide

ax = sns.barplot(data=top_20_votes, x='original_title', y='vote_average', 
                 palette = ['#25a6e0', '#25a6e0', '#25a6e0', 'black', 'black', '#25a6e0', 'black', 'black', 'black', 'black',
                           'black', 'black', 'black', '#25a6e0', '#25a6e0', '#25a6e0', '#25a6e0', '#25a6e0', '#25a6e0', '#25a6e0'])

sns.set(rc={'axes.facecolor':(0,0,0,0), 'figure.facecolor':(0,0,0,0)})
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right", rotation_mode='anchor')
ax.set_xlabel('Movie Titles')
ax.set_ylabel('Avg Rating (>10K Counts)')
ax.set_title('Highest Rated Movies from TMDB')

plt.savefig('./zippedData/highest rated movies from TMDB action adv', bbox_inches='tight', dpi=300);

In [ ]:
# code for plot used on drama slide

ax = sns.barplot(data=top_20_votes, x='original_title', y='vote_average', palette = ['black', 'black', '#4fb974', '#4fb974', '#4fb974', 'black', '#4fb974', '#4fb974', '#4fb974', '#4fb974',
                                                                                    '#4fb974', 'black', '#4fb974', 'black', 'black', '#4fb974', '#4fb974', 'black', 'black', '#4fb974'])

sns.set(rc={'axes.facecolor':(0,0,0,0), 'figure.facecolor':(0,0,0,0)})
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right", rotation_mode='anchor')
ax.set_xlabel('Movie Titles')
ax.set_ylabel('Avg Rating (>10K Counts)')
ax.set_title('Highest Rated Movies from TMDB')

plt.savefig('./zippedData/highest rated movies from TMDB drama', bbox_inches='tight', dpi=300);

In [ ]:
# code for comedy plot

ax = sns.barplot(data=top_20_votes, x='original_title', y='vote_average', palette = ['black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'magenta', 'black',
                                                                                    'magenta', 'magenta', 'magenta', 'black', 'magenta', 'black', 'black', 'black', 'black', 'black'])

sns.set(rc={'axes.facecolor':(0,0,0,0), 'figure.facecolor':(0,0,0,0)})
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right", rotation_mode='anchor')
ax.set_xlabel('Movie Titles')
ax.set_ylabel('Avg Rating (>10K Counts)')
ax.set_title('Highest Rated Movies from TMDB')

plt.savefig('./zippedData/highest rated movies from TMDB comedy', bbox_inches='tight', dpi=300);

# TMDB genre codes sampling

MOVIE
- Action          28
- Adventure       12
- Animation       16
- Comedy          35
- Crime           80
- Documentary     99
- Drama           18
- Family          10751
- Fantasy         14
- History         36
- Horror          27
- Music           10402
- Mystery         9648
- Romance         10749
- Science Fiction 878
- TV Movie        10770
- Thriller        53
- War             10752
- Western         37

In [ ]:
top_votes_adventure = top_votes['genre_ids'].str.replace('12', 'Adventure')
top_votes_adv_drama = top_votes_adventure.str.replace('18', 'Drama')
top_votes_adv_drama_act = top_votes_adv_drama.str.replace('28', 'Action')
top_votes_adv_drama_act_com = top_votes_adv_drama_act.str.replace('35', 'Comedy')
top_votes_adv_drama_act_com

In [ ]:
top_votes['Selected Genres'] = top_votes_adv_drama_act_com
top_votes.head(20)

In [ ]:
most_popular = top_vote_avg.loc[(top_vote_avg['popularity'] > 40) & (top_vote_avg['vote_count'] > 1000)]
most_popular = most_popular.sort_values('popularity', ascending=False)

In [ ]:
most_popular_adventure = most_popular['genre_ids'].str.replace('12', 'Adventure')
most_popular_adv_drama = most_popular_adventure.str.replace('18', 'Drama')
most_popular_adv_drama_act = most_popular_adv_drama.str.replace('28', 'Action')
most_popular_adv_drama_act_com = most_popular_adv_drama_act.str.replace('35', 'Comedy')
most_popular['Selected Genres'] = most_popular_adv_drama_act_com

In [ ]:
most_popular = most_popular.drop_duplicates(subset='title')

In [ ]:
most_popular_twenty = most_popular[:20]
most_popular_twenty

In [ ]:
# code for TMDB 'popularity' stats - ALL in action/adventure genre
ax = sns.barplot(data=most_popular_twenty, x='original_title', y='popularity', color='#25a6e0')

sns.set(rc={'axes.facecolor':(0,0,0,0), 'figure.facecolor':(0,0,0,0)})
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right", rotation_mode='anchor')
ax.set_xlabel('Movie Titles')
ax.set_ylabel('Popularity Rating on TMDB')
ax.set_title('Most Popular Movie Titles from TMDB')
plt.savefig('./zippedData/most popular movie titles from TMDB', bbox_inches='tight', dpi=300);

# The Numbers database (more budget info)

Juan cleaned worldwide_gross column and sorted it

I repeated it for my own learning, and was able to work around a Python error specific to Windows, needing to designate 'int64' for changing data type.  

In [ ]:
df_tn_budg = pd.read_csv('./zippedData/tn.movie_budgets.csv.gz')
df_tn_budg.index

In [ ]:
df_tn_budg.head()

In [ ]:
type(df_tn_budg['worldwide_gross'][1])

In [ ]:
df_tn_budg['worldwide_gross'] = df_tn_budg['worldwide_gross'].str.replace(',','')
df_tn_budg['worldwide_gross'] = df_tn_budg['worldwide_gross'].str.replace('$','')
df_tn_budg['worldwide_gross']

In [ ]:
df_tn_budg['worldwide_gross'].isna().sum()

In [ ]:
df_tn_budg ['worldwide_gross'] = df_tn_budg['worldwide_gross'].astype('int64')
df_tn_budg.head()

In [ ]:
type(df_tn_budg['worldwide_gross'][1])

In [ ]:
df_tn_budg.sort_values('worldwide_gross', ascending=False).head(11)

# Video Game Movies

Juan found a CSV filed from Kaggle on [film adaptations of video games](https://www.kaggle.com/datasets/bcruise/film-adaptations-of-video-games).  Double checked it to Wikipedia table as well.  Sorted by worldwide box office and created illustration for general understanding of box office info on video game films to date

In [ ]:
df_vid_game = pd.read_csv('./zippedData/video_game_films.csv')
df_vid_game.info

In [ ]:
df_vid_game.head()

In [ ]:
vid_games_box_off = df_vid_game.sort_values('Worldwide box office', ascending=False)
vid_games_box_off[:25]

In [ ]:
vid_games_plt_data = vid_games_box_off[:20]
vid_games_plt_data

In [ ]:
sns.set_theme(style='ticks')
sns.catplot(x='Worldwide box office',y='Title',kind='bar',data=vid_games_plt_data)
figsize=(15,6)
plt.show();

In [ ]:
ax = sns.barplot(data=vid_games_plt_data, x='Title', y=vid_games_plt_data['Worldwide box office'] / 1000000)

sns.set(rc={'axes.facecolor':(0,0,0,0), 'figure.facecolor':(0,0,0,0)})
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right", rotation_mode='anchor')
ax.set_xlabel('Movie Titles')
ax.set_ylabel('Worldwide Box Office \n ($ millions)')
ax.set_title('Video Game Movies Worldwide Box Office')
plt.savefig('./zippedData/video game movies worldwide box office', bbox_inches='tight', dpi=300);

In [ ]:
pwd

In [ ]:
ls

In [ ]:
cd zippedData/

In [ ]:
ls

In [ ]:
cd ..